In [ ]:
from tqdm import tqdm
import os
from datasets import load_dataset
import json
import nltk
import re
import difflib
from copy import deepcopy

import asyncio
import math
import json
import numpy as np
import random
import pandas as pd
from ollama import AsyncClient
import json

/root/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def save_to_json(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

In [295]:
from datasets import load_dataset

task_name = 'AgriExam'

dataset_name = "parquet"
data_files = {
    "dev": "/workdir/important_datasets/AGRIVQA_light/"+task_name+"/dev-00000-of-00001.parquet",
    "test": "/workdir/important_datasets/AGRIVQA_light/"+task_name+"/test-00000-of-00001.parquet",
    "validation": "/workdir/important_datasets/AGRIVQA_light/"+task_name+"/validation-00000-of-00001.parquet"
}
split = "validation"

dataset = load_dataset(dataset_name,data_files=data_files)

In [296]:
dataset

DatasetDict({
    dev: Dataset({
        features: ['id', 'question', 'options', 'explanation', 'image_1', 'image_2', 'image_3', 'image_4', 'image_5', 'img_type', 'answer', 'topic_difficulty', 'question_type', 'subfield', 'metadata'],
        num_rows: 16
    })
    test: Dataset({
        features: ['id', 'question', 'options', 'explanation', 'image_1', 'image_2', 'image_3', 'image_4', 'image_5', 'img_type', 'answer', 'topic_difficulty', 'question_type', 'subfield', 'metadata'],
        num_rows: 624
    })
    validation: Dataset({
        features: ['id', 'question', 'options', 'explanation', 'image_1', 'image_2', 'image_3', 'image_4', 'image_5', 'img_type', 'answer', 'topic_difficulty', 'question_type', 'subfield', 'metadata'],
        num_rows: 5613
    })
})

In [297]:
from datasets import load_dataset, concatenate_datasets

# Load dataset with splits

# Combine (concatenate) splits into a single dataset
combined_dataset = concatenate_datasets([dataset['dev'], dataset['test'], dataset['validation']])

# Now combined_dataset contains data from all splits
print(combined_dataset)


Dataset({
    features: ['id', 'question', 'options', 'explanation', 'image_1', 'image_2', 'image_3', 'image_4', 'image_5', 'img_type', 'answer', 'topic_difficulty', 'question_type', 'subfield', 'metadata'],
    num_rows: 6253
})


In [299]:
dataset=combined_dataset
dataset

Dataset({
    features: ['id', 'question', 'options', 'explanation', 'image_1', 'image_2', 'image_3', 'image_4', 'image_5', 'img_type', 'answer', 'topic_difficulty', 'question_type', 'subfield', 'metadata'],
    num_rows: 6253
})

In [375]:
client_ollama = AsyncClient(
  host='http://192.168.1.5:11434',
)

async def chat(content,image_path):
    message = {'role': 'user', 'content': content}
    if image_path != None:
        message.update({'images': [image_path]})
    response = await client_ollama.chat(model='llama3.2-vision', messages=[message],options={"temperature":0.2})
    return response['message']['content']

In [202]:
print(get_prompt(dataset[10],prompt))

[Question]
Propagation Method for cashew nut crop?
Options: ['Gooty', 'H-budding', 'T-budding', 'Soft wood grafting', 'All of these']
[End of Question]
Assign one category to the previous question. This question is taken from an agronomy exam, so try to be more specific than 'Agriculture' if it is a subfield. Your answer should start with one line containing only the category name.


In [284]:
from random import sample
ds=sample(list(dataset),2000)

In [264]:
# , so try to be more specific than 'Agriculture' if it is a subfield.

In [271]:
prompt = '''Try to assign one category from the list of categories to the previous question, if the question is not related to any of them create a new category. 
This question is taken from an agronomy exam.
Your answer should start with one line containing only the category name.'''

In [336]:
def get_prompt(doc,prompt,categories=None):
    question=doc['question']
    options=doc['options']
    if not categories:
        return '[Question]\n'+question+'\nOptions: '+options+'\n[End of Question]\n'+prompt
    return '[Question]\n'+question+'\nOptions: '+options+'\n[End of Question]\n'+'[Categories]\n'+categories+'\n[End of Categories]\n'+prompt

In [277]:
categories = {
    'Agricultural Engineering',
    'Animal Science',
    'General Agriculture',
    'Agricultural Genetics',
    'Agricultural Health',
    'Agricultural History',
    'Plant Breeding',
    'Soil Science',
    'Agricultural Statistics',
    'Wordplay and Analogies',
    'Economics',
    'Geography and History',
    'Geology',
    'Geometry',
    'Health Science',
    'Leadership and Management',
    'Math Concepts',
    'Mathematics',
    'Sports Knowledge'
}


In [323]:
categories = {'Agricultural Economics',
 'Agricultural Education',
 'Agricultural Engineering',
 'Agricultural Genetics',
 'Agricultural Geography',
 'Agricultural Health',
 'Agricultural History',
 'Agricultural Mathematics',
 'Agricultural Science',
 'Agricultural Statistics',
 'Anatomy',
 'Animal Science',
 'Biochemistry',
 'Economics',
 'English Language',
 'Environmental Science',
 'General Agriculture',
 'General Knowledge',
 'Geography and History',
 'Geology',
 'Geometry',
 'Health Science',
 'Leadership and Management',
 'Math Concepts',
 'Mathematics',
 'Plant Biology',
 'Plant Breeding',
 'Social Stratification',
 'Soil Science',
 'Sports Knowledge',
 'Wordplay and Analogies'}

In [ ]:

#categories = set()
for doc in tqdm(dataset):
    if not categories:
        string_categories='[]'
    else:
        string_categories=str(list(categories))
    response = await chat(get_prompt(doc,prompt,string_categories),None)
    try:
        category = response.split('\n')[0]
        if category not in categories:
            print(f"Category: {category} | Question: {doc['question']} | Options: {doc['options']}")
        categories.add(category)
    except:
        print('Gnappows')

  4%|▎         | 230/6253 [00:32<14:16,  7.03it/s]

Category: Environmental Science | Question: Ozone concentration in the atmosphere is reduced by? | Options: ['CO2 and O2', 'H2S and CH4', 'CFC and Cl2', 'N2 and N2O']


100%|██████████| 6253/6253 [15:17<00:00,  6.82it/s]


In [324]:
from collections import defaultdict
categories_dict = defaultdict(list)

for doc in tqdm(dataset):
    string_categories=str(list(categories))
    response = await chat(get_prompt(doc,prompt,string_categories),None)
    try:
        category = response.split('\n')[0]
        if category not in categories:
            print(f"Category: {category} | Question: {doc['question']} | Options: {doc['options']}")
        else:
            categories_dict[category].append(doc)
    except:
        print('Gnappows')

 41%|████      | 2555/6253 [06:16<09:07,  6.76it/s]

Category: Computer Security | Question: Which of the following is a threat for electronic payment systems? | Options: ['Computer Virus', 'Computer Worms', 'Trojan Horses', 'All of the above']


100%|██████████| 6253/6253 [15:18<00:00,  6.81it/s]


In [332]:
for key in categories_dict:
    print(key,len(categories_dict[key]))
    print()
    for i in range(3):
        print(categories_dict[key][i]['question'])
    print()

General Knowledge 56

Japanese style of gardens also known as?
FOB in international trade stands for?
In Indians context meaning of the term extension’ is transfer of technology while in USA it is?

General Agriculture 1240

Identify the correct order of the following methods of irrigation as per their water-use efficiency from the maximum to minimum efficiency.?
The insect, which attacks potato both in field as well as in storage is?
Disc Harrow which type tillage instrument?

Animal Science 167

A ______ dry matter intake100 kg body weight is recommended for breeding bull?
Which Fish has the highest protein content?
What is the Average Gestation period of buffalo? / 

Soil Science 394

Which of the following is the CORRECT sequence of soil-water erosion?
What is the total number of soil orders?
The most important feature of soil taxonomy is that it is based upon?

Plant Breeding 967

Synthetic seeds are…..?
Sundaram is a very high yielding clone of?
A variety developed by mixing the 

In [373]:
prompt_agronomy = '''Evaluate the exam question provided and determine if it pertains to the field of agronomy. Please begin your response with a single word: 'Yes' if the question pertains to or is relevant in the context of agronomy, or 'No' if it is not.'''

In [ ]:
dataset_light = list(dataset)[:200]

In [374]:
agronomy_dict = {'Yes': [], 'No': []}

for doc in tqdm(dataset):
    response = await chat(get_prompt(doc, prompt_agronomy),None)
    if response.lower().startswith('yes'):
        agronomy_dict['Yes'].append(doc)
    elif response.lower().startswith('no'):
        agronomy_dict['No'].append(doc)        
    else:
        print(f"Question: {doc['question']} | Options: {doc['options']}")

100%|██████████| 6253/6253 [32:00<00:00,  3.26it/s] 


In [377]:
print(len(agronomy_dict['No']))

1670


In [384]:
no = sample(agronomy_dict['No'], 40)
for doc in no:
    print(f"Question: {doc['question']} | Options: {doc['options']}")

Question: Which one of the following compounds forms the backbone of fats and oils? | Options: ['Glycerol', 'Glucose', 'Palmitiacid', 'Amino alcohol']
Question: National Botanical Research Institute is located at? | Options: ['New Delhi', 'Lucknow', 'Vadodara', 'Chennai']
Question: Blood group system was discovered by? | Options: ['Camillo Golgi', 'Ernst Haeckel', 'Thomas Cooley', 'Karl Landsteiner']
Question: “The first task of this Assembly is to free India through a new Constitution, to feed the starving people and to cloth the naked masses and to give every Indian the fullest opportunity to develop himself according to his capacity.” This quotation was given by? | Options: ['Pandit Jawaharlal Nehru', 'Dr. B. R. Ambedkar', 'Dr. S. Radha Krishnan', 'Dr. Rajendra Prasad']
Question: Nucleic acids are? | Options: ['Micro molecular compounds', 'Macro molecular compounds', 'Micro and macro molecular compounds', 'Non-molecular compounds']
Question: Watershed is a? | Options: ['Hydrological

In [383]:
prompt_categories = '''Try to assign to the previous question one category from the list of categories, if the question is not related to any of them create a new category. 
This question is taken from an agronomy exam.
Your answer should start with one line containing only the category name.'''

In [386]:
categories = {'Agricultural Economics',
 'Agricultural Education',
 'Agricultural Engineering',
 'Agricultural Genetics',
 'Agricultural Geography',
 'Agricultural Health',
 'Agricultural History',
 'Agricultural Mathematics',
 'Agricultural Science',
 'Agricultural Statistics',
 'Anatomy',
 'Animal Science',
 'Biochemistry',
 'Economics',
 'English Language',
 'Environmental Science',
 'General Agriculture',
 'General Knowledge',
 'Geography and History',
 'Geology',
 'Geometry',
 'Health Science',
 'Leadership and Management',
 'Math Concepts',
 'Mathematics',
 'Plant Biology',
 'Plant Breeding',
 'Social Stratification',
 'Soil Science',
 'Sports Knowledge',
 'Wordplay and Analogies'}

In [398]:
categories = {
    'Agricultural Economics',
    'Agricultural Education',
    'Agricultural Engineering',
    'Genetics and Breeding',
    'Sustainable Farming',
    'Agricultural Health',
    'Agricultural History',
    'Agricultural Geography',
    'Botany',
    'Data Science in Agriculture',
    'Climate and Agriculture',
    'Animal Science',
    'Biochemistry in Agriculture',
    'Environmental Economics',
    'Farm Management',
    'Food Science',
    'Irrigation and Water Management',
    'Plant Protection',
    'Precision Agriculture',
    'Rural Sociology',
    'Soil Science',
    'Crop Production',
    'Weed and Pest Management'
}

In [396]:
from collections import defaultdict

#categories = set()
categories_dict = defaultdict(list)

for doc in tqdm(agronomy_dict['Yes']):
    if not categories:
        string_categories='[]'
    else:
        string_categories=str(list(categories))
    response = await chat(get_prompt(doc,prompt_categories,string_categories),None)
    try:
        category = response.split('\n')[0]
        if category not in categories:
            print(f"Category: {category} | Question: {doc['question']} | Options: {doc['options']}")
        #categories.add(category)
        categories_dict[category].append(doc)
    except:
        print('Gnappows')

  5%|▌         | 245/4583 [00:34<10:41,  6.76it/s]

Category: Food Science in Agriculture | Question: The Indian name for clarified butterfat is…? | Options: ['(A) cream', '(B) buttermilk', '(C) dahi', '(D) ghee']


  6%|▌         | 264/4583 [00:36<09:52,  7.29it/s]

Category: Agriculture | Question: Golden revolution is related to? | Options: ['Dairy', 'Pulses', 'Agriculture', 'Horticulture']


 18%|█▊        | 839/4583 [01:55<08:18,  7.52it/s]

...

 91%|█████████ | 4166/4583 [09:38<00:59,  7.06it/s]

Category: Plant Physiology | Question: Study of the plant processes is called? | Options: ['Biotechnology', 'Plant physiology', 'Plant molecular biology', 'Ecology']


 94%|█████████▍| 4324/4583 [10:01<00:35,  7.31it/s]

Category: Plant Anatomy | Question: The cork in dicotyledonous plants is formed by? | Options: ['Phellogen', 'Phelloderm', 'Phellum', 'Cambium']


100%|██████████| 4583/4583 [10:36<00:00,  7.20it/s]


In [407]:
for key in categories_dict:
        
        print(key, len(categories_dict[key]))

Irrigation and Water Management 152
Animal Science 167
Soil Science 633
Plant Protection 503
Genetics and Breeding 927
Agricultural Engineering 81
Geography in Agriculture 43
Climate and Agriculture 56
Crop Production 1296
Agricultural Economics 170
Agricultural Health 5
Data Science in Agriculture 19
Biochemistry in Agriculture 152
Agricultural History 81
Environmental Economics 7
Precision Agriculture 19
Weed and Pest Management 111
Food Safety 14
Agricultural Education 45
Farm Management 54
Sustainable Farming 25
Food Science in Agriculture 1
Agriculture 1
Plant Anatomy 5
Plant Physiology 7
Community Ecology 1
Agricultural Geography 1
Rural Sociology 4
Botany 1
Crop Ecology 1
Plant Biology 1


In [406]:
for c in categories_dict['Data Science in Agriculture']:
    #agronomy_dict['Yes'].remove(c)
    print(c['question'])

The value of r (correlation coefficient) ranges between?
Which test is used to test the significance of the difference between two means?
Which of the following is ideal measure of disperson?
The correlation coefficient is used to determine?
Coefficient of variation is expressed as percentage of?
Analysis of variance is a statistical method of comparing the of several populations?
What will be the error d.f. for a R.B.D. field experiment with 8 treatments and 3 replications?
Comprehension:
The table given here shows the production of five types of tractors by a company in the year 2012 to 2017. Study the table and answer the questions that follow
Table: Production of Tractors by a Company
Table:
| Company /Year | 2012 | 2013 | 2014 | 2015 | 2016 | 2017 | Total |
|---------------|------|------|------|------|------|------|-------|
| A | 8 | 20 | 16 | 17 | 21 | 6 | 88 |
| B | 16 | 10 | 14 | 12 | 12 | 14 | 78 |
| C | 21 | 17 | 16 | 15 | 13 | 8 | 90 |
| D | 4 | 6 | 10 | 16 | 20 | 31 | 87 |
